In [1]:
import pandas as pd
import pandas
import seaborn as sns
import pyodbc
from gensim import corpora, models, similarities
import nltk
import re
import binascii
import logging, gensim, bz2
from nltk.tag import pos_tag
import logging
import re
import tempfile
import tarfile, zipfile
from gensim.utils import tokenize
from collections import defaultdict
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import vertica_python

tenant_id = 289
unuseful_list = 'for a of the and to in is very i my on it was as he she they us this hello hi over';
search_comment = "excellent";
#search_comment = "bad";


frequency = defaultdict(int)

conn_info = {'host': 'vertica04.reputation.ec2', 
             'port': 5433, 
             'user': 'rep_engine_user', 
             'password': 'r3p_3ng1n3', 
             'database': 'rwarehouse'}

#cnxn = pyodbc.connect("DSN=rwarehouse")
cnxn = vertica_python.connect(**conn_info)

#cnxn = pyodbc.connect("DSN=rwarehouse")
%matplotlib inline
sample_size = 10000

/Users/zzhao/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
query = """
SELECT
    tenants.name,
    locations.tenant_id,
    ratings.comment
FROM
    r4e_mongo.mongo_repbiz_location locations,
    r4e_mongo.mongo_repbiz_location_url urls,
    r4e_mongo.mongo_repbiz_rating ratings,
    r4e_mongo.mongo_repbiz_tenant_configurations tenants 
WHERE
    urls.id = locations.id 
    AND ratings.url = urls.url 
    AND locations.tenant_id = tenants.id
    AND (ratings.comment IS NOT NULL OR ratings.comment <> '.' OR ratings.comment <> ' ')
    AND locations.tenant_id = 289
    AND (ratings.source <> '%CARS%' OR ratings.source <> '%SURVEY%' OR ratings.source <> '%KIOSK%')
    AND tenants.active = 'T'
limit 10000
"""

result = pd.read_sql(query, cnxn)

#result

In [3]:
comments = result['comment'].values.tolist();
stoplist = set(unuseful_list.split());

filtered_comment = [[word for word in re.split(r'[;,.!*?~+\r\n\s\' ]\s*', comment.lower()) if word not in stoplist]
      for comment in comments]


for indi_comment in filtered_comment:
    #print(indi_comment)
    #delete all the very short(unuseful) comments
    if len(indi_comment) < 3:
        filtered_comment.remove(indi_comment);
        
for indi_comment in filtered_comment:
    # remove words that appear only once
    for word in indi_comment:
        #pprint(indi_comment)
        frequency[word] +=1;


        #dictionary = corpora.Dictionary.load(result)

In [4]:
from pprint import pprint
pprint(frequency)
filtered_comment = [[token for token in indi_comment if frequency[token] > 1]
        for indi_comment in filtered_comment]

defaultdict(<type 'int'>, {u'': 13874, u'raining': 3, u'car-lot': 1, u'cussed': 1, u'courtesous': 1, u'service-(only': 1, u'yellow': 4, u'shauntia': 1, u'four': 56, u'nalley': 1, u'hanging': 7, u'woody': 7, u'sonju': 25, u'manager-raymond': 3, u'towns': 1, u'payoff': 2, u'looking': 681, u'xts': 1, u'augustine': 4, u'vehiculo': 1, u'eligible': 6, u'snowing': 3, u'unanswered': 2, u'skipping': 1, u'refunding': 1, u'lori': 2, u'lord': 3, u'thomasville': 1, u'balise': 9, u'lift-gate/hatch': 1, u'\xa0our': 2, u'"repair"': 1, u'digit': 1, u'\xa0"you': 1, u'citrus': 1, u'dynamic': 3, u'bergstrom': 1, u'regional': 3, u'dell': 6, u'_mark': 1, u'hixson': 1, u'cavernous': 1, u'foul': 2, u'tmake': 1, u'malfunctioned': 1, u'uncertain': 2, u'bringing': 45, u'differentiated': 1, u'commision': 1, u'recall/tsa': 1, u'toyota/honda': 1, u'coyle': 1, u'sturgis': 1, u'wednesday': 34, u'multi-point': 5, u'"repairing"': 1, u'vehical': 10, u'stars)': 2, u'stereotypical': 6, u'monopolized': 1, u'commented': 5, 

In [6]:
filtered_comment

[[u'only',
  u'complaint',
  u'wish',
  u'would',
  u'have',
  u'told',
  u'me',
  u'that',
  u'work',
  u'did',
  u'reset',
  u'all',
  u'settings',
  u'control',
  u'panel',
  u'spent',
  u'quite',
  u'few',
  u'frustrated',
  u'minutes',
  u'even',
  u'had',
  u'drive',
  u'back',
  u'have',
  u'them',
  u'show',
  u'me',
  u'how',
  u'reset',
  u'one',
  u'thing',
  u''],
 [u'service',
  u'great',
  u'have',
  u'traded',
  u'2013',
  u'flex',
  u'2015',
  u'f150',
  u''],
 [u'sales',
  u'experience',
  u'excellent',
  u'david',
  u'robert',
  u'dixon',
  u'should',
  u'be',
  u'commended',
  u'their',
  u'professionalism',
  u'have',
  u'not',
  u'had',
  u'any',
  u'experience',
  u'yet',
  u'with',
  u'vehicle',
  u'service',
  u'department',
  u''],
 [u'friendly',
  u'approachable',
  u'service',
  u'guys',
  u'took',
  u'time',
  u'explain',
  u'everything',
  u'how',
  u'much',
  u'time',
  u'would',
  u'take',
  u'actual',
  u'work',
  u'satisfactory',
  u''],
 [u'couldn',
  

In [7]:
dictionary = corpora.Dictionary(filtered_comment)
dictionary.save('ford.dict')

corpus = [dictionary.doc2bow(indi_comment) for indi_comment in filtered_comment]
corpora.MmCorpus.serialize('/tmp/ford.mm', corpus)


In [8]:
print(dictionary.token2id)

{u'': 1, u'raining': 6752, u'yellow': 5071, u'four': 537, u'hanging': 3391, u'woody': 3334, u'sonju': 5681, u'manager-raymond': 5626, u'payoff': 2950, u'looking': 397, u'eligible': 2207, u'snowing': 3759, u'unanswered': 5708, u'lori': 6774, u'lord': 5668, u'balise': 2743, u'\xa0our': 7520, u'dynamic': 7741, u'regional': 2821, u'dell': 2881, u'foul': 7476, u'uncertain': 4600, u'bringing': 207, u'scholar': 7188, u'demmer': 7733, u'wednesday': 1761, u'multi-point': 1113, u'vehical': 1569, u'stars)': 5726, u'stereotypical': 4634, u'straight': 719, u'carfax': 4899, u'gaskets': 258, u'corvette': 6896, u'miller': 1706, u'tires': 523, u'second': 170, u'currie': 5594, u'melvin': 7827, u'feck': 6081, u'forgetting': 7028, u'interrnet': 6277, u'fingers': 1737, u'"sale"': 1555, u'\xa0thanks': 6148, u'barnette': 8086, u'increasing': 2150, u'shocks': 3489, u'specialist': 6498, u'avery': 8158, u'ravenel': 8018, u'here': 619, u'reported': 3136, u'jarrod': 6783, u'hers': 3827, u'affiliated': 6554, u'chi

In [9]:
corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 2),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 2),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 2),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1)],
 [(1, 1),
  (6, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1)],
 [(1, 1),
  (6, 1),
  (10, 1),
  (33, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 2),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1)],
 [(1, 1),
  (8, 1),
  (11, 1),
  (24, 1),
  (33, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 2),
  (63, 1),
  (64, 1)],
 [(1, 1),
  (6, 1),
  (17, 1),
  (33, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1)],
 [(1, 1),
  (33, 1),
  (51, 1),
  (72, 1),
  (73, 1),
  

In [10]:
lsi = models.LsiModel(corpus, id2word = dictionary, num_topics = 2)

In [11]:
#doc = corpus[1]
doc = dictionary.doc2bow(search_comment.lower().split());
#print(doc)
# lsi = latent semantic indexing
vec_lsi = lsi[doc]
type(doc)
#pprint(vec_lsi)

list

In [12]:
index = similarities.MatrixSimilarity(lsi[corpus])

In [13]:
type(index)

gensim.similarities.docsim.MatrixSimilarity

In [16]:
#perform similarity query against the corpus
sims = index[vec_lsi]
#pprint(doc[1])
#pprint(dictionary.get(522))


In [17]:
print(list(enumerate(sims)))
#check because there are way too many similar ones if count ~1 are very similar

[(0, 0.97653186), (1, 0.54690742), (2, 0.91943735), (3, 0.68980056), (4, 0.80642462), (5, 0.90449548), (6, 0.9980588), (7, 0.72993129), (8, 0.70436722), (9, 0.99996638), (10, 0.98143125), (11, 0.66665131), (12, 0.97821081), (13, 0.85331047), (14, 0.42916113), (15, 0.93438685), (16, 0.96449995), (17, 0.74157625), (18, 0.99360007), (19, 0.31184363), (20, 0.98452932), (21, 0.79630721), (22, 0.26861385), (23, 0.26054311), (24, 0.98412198), (25, 0.91354239), (26, 0.82849813), (27, 0.68440801), (28, 0.99377143), (29, 0.87208247), (30, 0.90805984), (31, 0.99640095), (32, 0.74029624), (33, 0.9994688), (34, 0.98297203), (35, 0.93406391), (36, 0.56849939), (37, 0.99259043), (38, 0.82840455), (39, 0.72840619), (40, 0.89664274), (41, 0.40835947), (42, 0.87255919), (43, 0.99607635), (44, 0.98880243), (45, 0.7743907), (46, 0.35849857), (47, 0.98870599), (48, 0.35200942), (49, 0.95424366), (50, 0.32462972), (51, 0.8145085), (52, 0.65588641), (53, 0.31153274), (54, 0.99517405), (55, 0.88461173), (56, 

In [18]:
# sort by descending order
sims = sorted(enumerate(sims), key = lambda item: -item[1])
print(sims)

[(1265, 1.0), (2555, 1.0), (3354, 1.0), (4127, 1.0), (5210, 1.0), (5293, 1.0), (5306, 1.0), (6409, 1.0), (912, 0.99999994), (6286, 0.99999994), (7213, 0.99999988), (5909, 0.99999982), (7866, 0.99999982), (3004, 0.99999976), (8468, 0.99999976), (434, 0.9999997), (6690, 0.9999997), (7722, 0.9999997), (3992, 0.99999964), (4032, 0.99999958), (4768, 0.99999958), (3839, 0.99999952), (4277, 0.99999952), (3380, 0.9999994), (5788, 0.99999917), (963, 0.99999899), (9288, 0.99999881), (3392, 0.99999869), (2240, 0.99999857), (1492, 0.99999797), (2620, 0.99999797), (6429, 0.99999756), (2083, 0.9999975), (7596, 0.99999726), (7850, 0.99999714), (5603, 0.99999702), (4716, 0.9999969), (7238, 0.99999678), (6417, 0.99999577), (4732, 0.99999553), (3127, 0.99999499), (8500, 0.9999944), (9019, 0.9999944), (7135, 0.9999941), (7145, 0.9999941), (2999, 0.99999374), (4972, 0.99999368), (4607, 0.99999344), (5428, 0.99999344), (5604, 0.99999279), (4582, 0.99999237), (4591, 0.99999201), (5972, 0.99999201), (8586, 0

In [23]:
filtered_comment[3354]

[u'we',
 u'bought',
 u'used',
 u'vehicle',
 u'from',
 u'dealer',
 u'promised',
 u'few',
 u'items',
 u'be',
 u'fixed',
 u'or',
 u'replaced',
 u'before',
 u'we',
 u'actually',
 u'bought',
 u'truck',
 u'things',
 u'that',
 u'were',
 u'promised',
 u'were',
 u'either',
 u'not',
 u'done',
 u'proceeded',
 u'say',
 u'that',
 u'were',
 u'or',
 u'were',
 u'done',
 u'am',
 u'not',
 u'happy',
 u'with',
 u'how',
 u'nice',
 u'many',
 u'managers',
 u'sales',
 u'representatives',
 u'are',
 u'when',
 u'your',
 u'looking',
 u'buy',
 u'car',
 u'but',
 u'when',
 u'you',
 u'buy',
 u'are',
 u'completely',
 u'rude',
 u'liars',
 u'would',
 u'not',
 u'recommend',
 u'anyone',
 u'buy',
 u'car',
 u'from',
 u'pacifico',
 u'ford',
 u'']

In [34]:
#top_10_similar = [];
index = 0;
while index < 20:
    print("Comment: "+ str(filtered_comment[sims[index][0]]));
    print("Similarity: " + str.format('{0:.15f}', sims[index][1]) + "\n");
    #print(sims[index]);
    index += 1;

Comment: [u'took', u'2013', u'lincoln', u'navigator', u'service', u'have', u'only', u'had', u'service', u'three', u'times', u'since', u'purchased', u'vehicle', u'knowing', u'that', u'will', u'be', u'using', u'car', u'more', u'wanted', u'have', u'service', u'done', u'before', u'holiday', u'rush', u'happened', u'mitch', u'service', u'advisor', u'great', u'polite', u'updated', u'me', u'when', u'there', u'information', u'passed', u'along', u'him', u'only', u'issue', u'length', u'time', u'took', u'service', u'appointment', u'at', u'8:30', u'am', u'did', u'not', u'receive', u'vehicle', u'until', u'just', u'before', u'noon', u'had', u'known', u'going', u'take', u'long', u'would', u'have', u'make', u'other', u'arrangements', u'left', u'vehicle', u'there', u'next', u'time', u'will', u'ask', u'if', u'are', u'schedule', u'how', u'much', u'time', u'will', u'take', u'when', u'call', u'appointment', u'']
Similarity: 1.000000000000000

Comment: [u'although', u'did', u'not', u'have', u'what', u'we', u